In [ ]:
!pip list

In [ ]:
import json
from flask import Flask, request, jsonify, make_response
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
import jwt
from flask_cors import CORS
from datetime import datetime, timedelta
from functools import wraps

app = Flask(__name__)

app.config['SECRET_KEY'] = 'thisissecret'
app.config['SQLALCHEMY_DATABASE_URI'] = app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///todo.db'
bcrypt = Bcrypt(app)
db = SQLAlchemy(app)

CORS(app)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})


class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50))
    password = db.Column(db.String(80))
    admin = db.Column(db.Boolean)

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.String(50))
    complete = db.Column(db.Boolean)
    user_id = db.Column(db.Integer)
    
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    post = db.Column(db.String(50))
    user_id = db.Column(db.Integer)

def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None

        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            # data = jwt.decode(token, app.config['SECRET_KEY'])
            data = jwt.decode(token, app.config['SECRET_KEY'], algorithms=['HS256'])
            current_user = User.query.filter_by(id=data['id']).first()
            print("current_user = ", current_user)
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_user, *args, **kwargs)

    return decorated

@app.route('/api/user', methods=['GET'])
@token_required
def get_all_users(current_user):
#def get_all_users():
    print("JWT passed.")
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})
    users = User.query.all()
    output = []
    for user in users:
        user_data = {}
        user_data['id'] = user.id
        user_data['name'] = user.name
        # user_data['password'] = user.password
        user_data['admin'] = user.admin
        print("user_data(",user,")")
        output.append(user_data)
    print("output = ",output)
    return jsonify({'users' : output})

@app.route('/api/user/<id>', methods=['GET'])
@token_required
def get_one_user(current_user, id):

    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    user = User.query.filter_by(id=id).first()

    if not user:
        return jsonify({'message' : 'No user found!'})

    user_data = {}
    user_data['id'] = user.id
    user_data['name'] = user.name
    # user_data['password'] = user.password
    user_data['admin'] = user.admin
    print(user_data)
    return jsonify({'user' : user_data})

@app.route('/api/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(name=data['name'], password=hashed_password, admin=False)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'Registration completed!'})

@app.route('/api/user/<id>', methods=['POST'])
@token_required
def create_user(current_user, id):
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'}) 
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(name=data['name'], password=hashed_password, admin=False)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'New user created!'})

@app.route('/api/user/<id>', methods=['PUT'])
@token_required
def promote_user(current_user, id):
    print(current_user)
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    user = User.query.filter_by(id=id).first()

    if not user:
        return jsonify({'message' : 'No user found!'})

    user.admin = True
    db.session.commit()

    return jsonify({'message' : 'The user has been promoted!'})

@app.route('/api/user/<id>', methods=['DELETE'])
@token_required
def delete_user(current_user, id):
    if not current_user.admin:
           return jsonify({'message' : 'Cannot perform that function!'})

    user = User.query.filter_by(id=id).first()

    if not user:
        return jsonify({'message' : 'No user found!'})

    db.session.delete(user)
    db.session.commit()

    return jsonify({'message' : 'The user has been deleted!'})

@app.route('/api/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    user = User.query.filter_by(name=auth.username).first()

    if not user:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(user.password, auth.password):
        #token = jwt.encode({'id' : user.id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=30)}, app.config['SECRET_KEY'])
        token = jwt.encode({'id' : user.id, 'exp' : datetime.now() + timedelta(minutes=30)}, app.config['SECRET_KEY'])
      
        #return jsonify({'token' : token.decode('UTF-8')})
        return jsonify({'token' : token})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

@app.route('/api/todo', methods=['GET'])
@token_required
def get_all_todos(current_user):
    todos = Todo.query.filter_by(user_id=current_user.id).all()

    output = []

    for todo in todos:
        todo_data = {}
        todo_data['id'] = todo.id
        todo_data['text'] = todo.text
        todo_data['complete'] = todo.complete
        output.append(todo_data)
    print("output = ",output)
    return jsonify({'todos' : output})

@app.route('/api/todo/<todo_id>', methods=['GET'])
@token_required
def get_one_todo(current_user, todo_id):
    todo = Todo.query.filter_by(id=todo_id, user_id=current_user.id).first()

    if not todo:
        return jsonify({'message' : 'No todo found!'})

    todo_data = {}
    todo_data['id'] = todo.id
    todo_data['text'] = todo.text
    todo_data['complete'] = todo.complete

    return jsonify(todo_data)

@app.route('/api/todo', methods=['POST'])
@token_required
def create_todo(current_user):
    data = request.get_json()
    new_todo = Todo(text=data['text'], complete=False, user_id=current_user.id)
    db.session.add(new_todo)
    db.session.commit()

    return jsonify({'message' : "Todo created!"})

@app.route('/api/todo/<todo_id>', methods=['PUT'])
@token_required
def complete_todo(current_user, todo_id):
    data = request.get_json()
    print("data=",data)
    todo = Todo.query.filter_by(id=todo_id, user_id=current_user.id).first()
    if not todo:
        return jsonify({'message' : 'No todo found!'})
    todo.complete = data['complete']
    if todo.complete:
        msg =  'Todo item has been completed!'
    else:
        msg =  'Todo item has not been completed!'
    db.session.commit()
    return jsonify({'message' : msg})

@app.route('/api/todo/<todo_id>', methods=['DELETE'])
@token_required
def delete_todo(current_user, todo_id):
    todo = Todo.query.filter_by(id=todo_id, user_id=current_user.id).first()

    if not todo:
        return jsonify({'message' : 'No todo found!'})

    db.session.delete(todo)
    db.session.commit()

    return jsonify({'message' : 'Todo item deleted!'})

if __name__ == '__main__':
    app.run('localhost', port=9028)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask_sqlalchemy/__init__.py:868: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9028/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jan/2023 22:52:03] "PUT /api/todo/1 HTTP/1.1" 200 -


current_user =  <User 6>
data= {'complete': False}


127.0.0.1 - - [08/Jan/2023 22:52:24] "GET /api/todo HTTP/1.1" 200 -


current_user =  <User 6>
output =  [{'id': 3, 'text': 'Check the mail.', 'complete': True}]


...

127.0.0.1 - - [08/Jan/2023 22:54:45] "GET /api/todo HTTP/1.1" 200 -


current_user =  <User 7>
output =  [{'id': 4, 'text': 'Clean the appartment.', 'complete': False}, {'id': 5, 'text': 'Prepare for class.', 'complete': False}, {'id': 6, 'text': 'Clean the appartment.', 'complete': True}]


127.0.0.1 - - [08/Jan/2023 22:55:31] "GET /api/todo HTTP/1.1" 200 -


current_user =  <User 7>
output =  [{'id': 4, 'text': 'Clean the appartment.', 'complete': False}, {'id': 5, 'text': 'Prepare for class.', 'complete': False}, {'id': 6, 'text': 'Leave for vacation.', 'complete': True}]


127.0.0.1 - - [09/Jan/2023 17:59:54] "GET /api/login HTTP/1.1" 401 -
127.0.0.1 - - [09/Jan/2023 18:01:21] "GET /api/login HTTP/1.1" 401 -
127.0.0.1 - - [09/Jan/2023 18:02:01] "GET /api/login HTTP/1.1" 200 -
